start with sql  if user grow then move to nosql

no sql structure

In [ ]:
{
    "user_id": 1,
    "username": "Alice",
    "subjects": [
        {
            "subject_name": "Mathematics",
            "chapters": [
                {
                    "chapter_name": "Algebra",
                    "topics": ["Linear Equations", "Quadratic Equations"]
                }
            ]
        }
    ]
}

In [ ]:
from pymongo import MongoClient

client = MongoClient('mongodb://localhost:27017/')
db = client['library']
users_collection = db['users']

# CREATE
def create_user(username, subjects=None):
    user = {
        "username": username,
        "subjects": subjects or []
    }
    result = users_collection.insert_one(user)
    print(f"Created user: {username} (ID: {result.inserted_id})")

# Add subject with chapters and topics
def add_subject(user_id, subject_name, chapters=None):
    users_collection.update_one(
        {"_id": user_id},
        {"$push": {"subjects": {"subject_name": subject_name, "chapters": chapters or []}}}
    )

# READ
def display_library():
    for user in users_collection.find():
        print(f"User: {user['username']} (ID: {user['_id']})")
        for subject in user['subjects']:
            print(f"  Subject: {subject['subject_name']}")
            for chapter in subject['chapters']:
                print(f"    Chapter: {chapter['chapter_name']}")
                for topic in chapter['topics']:
                    print(f"      Topic: {topic}")

# Example usage
create_user("Alice")
add_subject(users_collection.find_one({"username": "Alice"})["_id"], "Mathematics", 
            [{"chapter_name": "Algebra", "topics": ["Linear Equations"]}])
display_library()

sql structures

In [ ]:
import sqlite3 # change it to mysql

# Connect to SQLite database
conn = sqlite3.connect('library.db')
cursor = conn.cursor()

In [ ]:
# Create tables for the hierarchical structure
def create_tables():
    # Users table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS users (
            user_id INTEGER PRIMARY KEY AUTOINCREMENT,
            username TEXT NOT NULL UNIQUE
        )
    ''')

    # Subjects table (child of users)
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS subjects (
            subject_id INTEGER PRIMARY KEY AUTOINCREMENT,
            subject_name TEXT NOT NULL,
            user_id INTEGER,
            FOREIGN KEY (user_id) REFERENCES users(user_id)
        )
    ''')

    # Chapters table (child of subjects)
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS chapters (
            chapter_id INTEGER PRIMARY KEY AUTOINCREMENT,
            chapter_name TEXT NOT NULL,
            subject_id INTEGER,
            FOREIGN KEY (subject_id) REFERENCES subjects(subject_id)
        )
    ''')

    # Topics table (child of chapters)
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS topics (
            topic_id INTEGER PRIMARY KEY AUTOINCREMENT,
            topic_name TEXT NOT NULL,
            chapter_id INTEGER,
            FOREIGN KEY (chapter_id) REFERENCES chapters(chapter_id)
        )
    ''')

    conn.commit()
    print("Database tables created successfully.")

In [ ]:
# CRUD Operations

# --- CREATE ---
def create_user(username):
    cursor.execute("INSERT OR IGNORE INTO users (username) VALUES (?)", (username,))
    conn.commit()
    cursor.execute("SELECT user_id FROM users WHERE username = ?", (username,))
    user_id = cursor.fetchone()[0]
    print(f"Created user: {username} (ID: {user_id})")
    return user_id

In [ ]:
def create_subject(user_id, subject_name):
    cursor.execute("INSERT INTO subjects (subject_name, user_id) VALUES (?, ?)", 
                  (subject_name, user_id))
    conn.commit()
    cursor.execute("SELECT subject_id FROM subjects WHERE subject_name = ? AND user_id = ?", 
                  (subject_name, user_id))
    subject_id = cursor.fetchone()[0]
    print(f"Created subject: {subject_name} for User ID {user_id}")
    return subject_id

In [ ]:
def create_chapter(subject_id, chapter_name):
    cursor.execute("INSERT INTO chapters (chapter_name, subject_id) VALUES (?, ?)", 
                  (chapter_name, subject_id))
    conn.commit()
    cursor.execute("SELECT chapter_id FROM chapters WHERE chapter_name = ? AND subject_id = ?", 
                  (chapter_name, subject_id))
    chapter_id = cursor.fetchone()[0]
    print(f"Created chapter: {chapter_name} for Subject ID {subject_id}")
    return chapter_id

In [ ]:
def create_topic(chapter_id, topic_name):
    cursor.execute("INSERT INTO topics (topic_name, chapter_id) VALUES (?, ?)", 
                  (topic_name, chapter_id))
    conn.commit()
    print(f"Created topic: {topic_name} for Chapter ID {chapter_id}")

In [ ]:
# --- READ ---
def display_library():
    print("\n=== Library Hierarchy ===")
    cursor.execute("SELECT * FROM users")
    users = cursor.fetchall()
    for user in users:
        user_id, username = user
        print(f"User: {username} (ID: {user_id})")
        
        cursor.execute("SELECT subject_id, subject_name FROM subjects WHERE user_id = ?", (user_id,))
        subjects = cursor.fetchall()
        for subject_id, subject_name in subjects:
            print(f"  Subject: {subject_name} (ID: {subject_id})")
            
            cursor.execute("SELECT chapter_id, chapter_name FROM chapters WHERE subject_id = ?", (subject_id,))
            chapters = cursor.fetchall()
            for chapter_id, chapter_name in chapters:
                print(f"    Chapter: {chapter_name} (ID: {chapter_id})")
                
                cursor.execute("SELECT topic_name FROM topics WHERE chapter_id = ?", (chapter_id,))
                topics = cursor.fetchall()
                for topic in topics:
                    topic_name = topic[0]
                    print(f"      Topic: {topic_name}")

In [ ]:
# --- UPDATE ---
def update_user(user_id, new_username):
    cursor.execute("UPDATE users SET username = ? WHERE user_id = ?", (new_username, user_id))
    conn.commit()
    print(f"Updated User ID {user_id} to username: {new_username}")

In [ ]:
def update_subject(subject_id, new_subject_name):
    cursor.execute("UPDATE subjects SET subject_name = ? WHERE subject_id = ?", 
                  (new_subject_name, subject_id))
    conn.commit()
    print(f"Updated Subject ID {subject_id} to: {new_subject_name}")

In [ ]:
def update_chapter(chapter_id, new_chapter_name):
    cursor.execute("UPDATE chapters SET chapter_name = ? WHERE chapter_id = ?", 
                  (new_chapter_name, chapter_id))
    conn.commit()
    print(f"Updated Chapter ID {chapter_id} to: {new_chapter_name}")

In [ ]:
def update_topic(topic_id, new_topic_name):
    cursor.execute("UPDATE topics SET topic_name = ? WHERE topic_id = ?", 
                  (new_topic_name, topic_id))
    conn.commit()
    print(f"Updated Topic ID {topic_id} to: {new_topic_name}")

In [ ]:
# --- DELETE ---
def delete_user(user_id):
    cursor.execute("DELETE FROM topics WHERE chapter_id IN (SELECT chapter_id FROM chapters WHERE subject_id IN (SELECT subject_id FROM subjects WHERE user_id = ?))", (user_id,))
    cursor.execute("DELETE FROM chapters WHERE subject_id IN (SELECT subject_id FROM subjects WHERE user_id = ?)", (user_id,))
    cursor.execute("DELETE FROM subjects WHERE user_id = ?", (user_id,))
    cursor.execute("DELETE FROM users WHERE user_id = ?", (user_id,))
    conn.commit()
    print(f"Deleted User ID {user_id} and all related data.")

In [ ]:
def delete_subject(subject_id):
    cursor.execute("DELETE FROM topics WHERE chapter_id IN (SELECT chapter_id FROM chapters WHERE subject_id = ?)", (subject_id,))
    cursor.execute("DELETE FROM chapters WHERE subject_id = ?", (subject_id,))
    cursor.execute("DELETE FROM subjects WHERE subject_id = ?", (subject_id,))
    conn.commit()
    print(f"Deleted Subject ID {subject_id} and all related data.")

In [ ]:
def delete_chapter(chapter_id):
    cursor.execute("DELETE FROM topics WHERE chapter_id = ?", (chapter_id,))
    cursor.execute("DELETE FROM chapters WHERE chapter_id = ?", (chapter_id,))
    conn.commit()
    print(f"Deleted Chapter ID {chapter_id} and all related topics.")

In [ ]:
def delete_topic(topic_id):
    cursor.execute("DELETE FROM topics WHERE topic_id = ?", (topic_id,))
    conn.commit()
    print(f"Deleted Topic ID {topic_id}")

In [ ]:
# Main function to test CRUD operations
def main():
    create_tables()

    # CREATE: Add sample data
    user_id = create_user("Alice")
    subject_id = create_subject(user_id, "Mathematics")
    chapter_id = create_chapter(subject_id, "Algebra")
    create_topic(chapter_id, "Linear Equations")
    create_topic(chapter_id, "Quadratic Equations")

    user_id2 = create_user("Bob")
    subject_id2 = create_subject(user_id2, "Physics")
    chapter_id2 = create_chapter(subject_id2, "Mechanics")
    create_topic(chapter_id2, "Newton's Laws")

    # READ: Display the library
    display_library()

    # UPDATE: Modify some entries
    update_user(user_id, "Alice_Updated")
    update_subject(subject_id, "Advanced Mathematics")
    update_chapter(chapter_id, "Advanced Algebra")
    cursor.execute("SELECT topic_id FROM topics WHERE chapter_id = ? LIMIT 1", (chapter_id,))
    topic_id = cursor.fetchone()[0]
    update_topic(topic_id, "Linear Systems")

    # READ: Display updated library
    print("\nAfter Updates:")
    display_library()

    # DELETE: Remove some entries
    delete_topic(topic_id)
    delete_chapter(chapter_id2)
    delete_subject(subject_id)
    delete_user(user_id2)

    # READ: Display final library
    print("\nAfter Deletions:")
    display_library()

    conn.close()

In [ ]:
# indexing 

# replication

# partition by rows

# horizontal partitioning  sharding

# vertical partitioning

# normalization

# denormalization




